In [1]:
from glob import glob

files = glob("augment/*")
len(files)

168

In [2]:
import json
import re

loaded_files: list[str] = []
to_check = []

def load_and_check(path: str):
	llm_output = open(path).read()

	# location_labels = []

	pattern = r'<ner>(.*?)</ner>'
	matches: list[str] = re.findall(pattern, llm_output, re.DOTALL)

	if len(matches) < 100:
		patch = []
	
		fmatches = [i for i in matches if "<ner>" in i]
		patch = [i for i in matches if "<ner>" not in i]

		# loaded_files.extend(nmatches)
		patch.extend(j.strip() for i in fmatches for j in i.split("<ner>"))
		if len(patch) == 100 or not fmatches:
			loaded_files.extend(patch)
		else:
			print(path)
			to_check.append(path)
			print(len(fmatches))
			for i in fmatches:
				print(i)
				print()
			print("\n")
	else:
		loaded_files.extend(matches)
	# for i in fmatches:
	# 	print(i)
	# print(len(matches))
# for match in matches:
# 	# Split the content into lines and remove leading/trailing whitespace and dashes
# 	pass

for file in files:
	load_and_check(file)

In [3]:
len(loaded_files)

16782

In [4]:
files[0]

'augment/4e36bcb0-3a6e-4765-818a-b1d113fa6613'

In [5]:
print(loaded_files[0].strip())

The O
American O
RedCross O
is O
providing O
relief O
efforts O
to O
the O
people O
of O
Puerto B-LOC
Rico I-LOC
after O
Hurricane O
Fiona O
made O
landfall O
. O
Our O
team O
in O
New B-LOC
York I-LOC
is O
also O
collecting O
donations O
for O
the B-LOC
victims O
of O
the O
disaster O


In [17]:
perfect_entities = []
tags = ["O", "B-LOC", "I-LOC"]
ponct = ".!,#"

def check_sentence(raw: tuple[str, str]):
	return ((len(raw) == 2) and (raw[1] in tags)) # or ((len(raw) == 1) and (raw[0] in ponct))

for sentence in loaded_files: # example
	example = []
	word_ner_list = sentence.strip().split("\n") # list of (word, ner)
	row_splts = [word_ner.split() for word_ner in word_ner_list]
	# len(i) == 2 and i[1] in tags
	if all(check_sentence(i) for i in row_splts):
		perfect_entities.append(sentence.strip().replace("the B-LOC", "the O"))
	# for word_ner in word_ner_list:
	# 	if not word_ner:
	# 		continue
	# 	row_splts = word_ner.split()

		# row_splts = [
		# 	row_splts[i: i + 2] for i in range(0, len(row_splts), 2)
		# ]
		# for i, row_splt in enumerate(row_splts):
		# 	flag = False
		# 	if len(row_splt) == 2:
		# 		lab = row_splt[1]
		# 		if lab[:4] in ["O", "B-LOC", "I-LOC"]:
		# 			if len(lab) <= 4:
		# 				example.append(row_splt)
					
		# 			flag = True
				
		# print(row_splts)
	# break

In [18]:
len(perfect_entities)

6030

In [19]:
import random

print(random.choice(perfect_entities))

Emergency O
workers O
are O
providing O
aid O
to O
thousands O
of O
people O
affected O
by O
a O
devastating O
hurricane O
that O
hit O
the O
Caribbean B-LOC
, I-LOC
specifically O
Puerto O
Rico's I-LOC
western I-LOC
coastline I-LOC
. O


In [23]:
sentences = "\n\n".join(perfect_entities)
with open("data/ExtratTrain.txt", "w") as f:
    f.write(sentences)